In [1]:
from aeon.datasets import load_italy_power_demand
X, y = load_italy_power_demand(return_type='numpy2d')

In [2]:
import numpy as np
from numba import njit

from aeon.classification.base import BaseClassifier
from aeon.distances import distance


class Node:

    def __init__(
        self,
        node_id: int,
        _is_leaf: bool,
        label=None,
        class_distribution=None,
        splitter=None,
    ):
        self.node_id = node_id
        self._is_leaf = _is_leaf
        self.label = label
        self.splitter = splitter
        self.class_distribution = class_distribution or {}
        self.children = {}

    def __repr__(self):

        # Print TreeNode

        if not self._is_leaf:
            message = self.node_id + " Internal node splits using distance measure " + str(self.splitter[1]) + " and the branches are " + str(list(self.children.keys()))
        else:
            message = self.node_id + " Leaf node with label " + str(self.label) + " and class distribution " + str(self.class_distribution)

        return message

class ProximityTree(BaseClassifier):

    def __init__(
        self,
        n_splitters: int = 5,
        max_depth: int = None,
        min_samples_split: int = 2,
        random_state: int = 0,
        n_jobs: int = 1,
        verbose: int = 0,
    ) -> None:
        self.n_splitters = n_splitters
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.verbose = verbose
        super().__init__()

    def get_parameter_value(self, X=None):
        """Generate random parameter values.

        For a list of distance measures, generate a dictionary
        of parameterized distances.

        Parameters
        ----------
        X : np.ndarray of shape (n_cases, n_timepoints)

        Returns
        -------
        distance_param : a dictionary of distances and their
        parameters.
        """
        X_std = X.std()
        param_ranges = {
            "euclidean": {},
            "dtw": {"window": (0, 0.25)},
            "ddtw": {"window": (0, 0.25)},
            "wdtw": {"g": (0, 1)},
            "wddtw": {"g": (0, 1)},
            "erp": {"g": (X_std / 5, X_std)},
            "lcss": {"epsilon": (X_std / 5, X_std), "window": (0, 0.25)},
        }
        random_params = {}
        for measure, ranges in param_ranges.items():
            random_params[measure] = {
                param: np.round(np.random.uniform(low, high), 3)
                for param, (low, high) in ranges.items()
            }
        # For TWE
        lmbda = np.random.randint(0, 9)
        exponent_range = np.arange(1, 6)  # Exponents from -5 to 1 (inclusive)
        random_exponent = np.random.choice(exponent_range)
        nu = 1 / 10**random_exponent
        random_params["twe"] = {"lmbda": lmbda, "nu": nu}

        # For MSM
        base = 10
        # Exponents from -2 to 2 (inclusive)
        exponents = np.arange(-2, 3, dtype=np.float64)
        # Randomly select an index from the exponent range
        random_index = np.random.randint(0, len(exponents))
        c = base ** exponents[random_index]
        random_params["msm"] = {"c": c}

        return random_params

    def get_candidate_splitter(self, X, y):
        """Generate candidate splitter.

        Takes a time series dataset and a set of parameterized
        distance measures to create a candidate splitter, which
        contains a parameterized distance measure and a set of exemplars.

        Parameters
        ----------
        X : np.ndarray shape (n_cases, n_timepoints)
            The training input samples.
        y : np.array shape (n_cases,) or (n_cases,1)
        parameterized_distances : dictionary
            Contains the distances and their parameters.

        Returns
        -------
        splitter : list of two dictionaries
            A distance and its parameter values and a set of exemplars.
        """
        _X = X
        _y = y

        exemplars = {}
        for label in np.unique(_y):
            y_new = _y[_y == label]
            X_new = _X[_y == label]
            id = np.random.randint(0, X_new.shape[0])
            exemplars[y_new[id]] = X_new[id, :]

        # Create a list with first element exemplars and second element a
        # random parameterized distance measure
        parameterized_distances = self.get_parameter_value(X)
        n = np.random.randint(0, 9)
        dist = list(parameterized_distances.keys())[n]
        splitter = [exemplars, {dist: parameterized_distances[dist]}]

        return splitter

    def _build_tree(
        self, 
        X, 
        y, 
        depth, 
        node_id,
        parent_target_value=None):

        # If the data reaching the node is empty
        if len(X) == 0:
            leaf_label = parent_target_value
            leaf_distribution = {}
            leaf = Node(node_id=node_id, _is_leaf=True, label=leaf_label, class_distribution=leaf_distribution)
            return leaf

        # Target value in current node
        target_value = self._find_target_value(y)
        class_distribution = {label: count / len(y) for label, count in zip(*np.unique(y, return_counts=True))}

        # If min sample splits is reached
        if (self.min_samples_split >= len(X)):
            leaf_label = target_value
            leaf = Node(node_id=node_id, _is_leaf=True, label = leaf_label, class_distribution = class_distribution)
            return leaf

        # If max depth is reached
        if (self.max_depth is not None) and (depth >= self.max_depth):
            leaf_label = target_value
            leaf = Node(node_id=node_id, _is_leaf=True, label=leaf_label, class_distribution = class_distribution)
            return leaf

        # Pure node
        if len(np.unique(y)) == 1:
            leaf_label = target_value
            leaf = Node(node_id=node_id, _is_leaf=True, label=leaf_label, class_distribution = class_distribution)
            return leaf

        # Find the best splitter
        splitter = self.get_best_splitter(X, y)

        # Create root node
        node = Node(node_id=node_id, _is_leaf=False, splitter=splitter)

        # For each exemplar split the data
        labels = list(splitter[0].keys())
        measure = list(splitter[1].keys())[0]
        X_child = [[] for _ in labels]
        y_child = [[] for _ in labels]
        for i in range(len(X)):
            min_dist = np.inf
            id = None
            for j in range(len(labels)):
                dist = distance(X[i], splitter[0][labels[j]],
                               metric = measure, kwargs = splitter[1][measure])
                if dist < min_dist:
                    min_dist = dist
                    id = j
            X_child[id].append(X[i])
            y_child[id].append(y[i])
        X_child = [np.array(ele) for ele in X_child]
        y_child = [np.array(ele) for ele in y_child]
        # For each exemplar, create a branch
        for i in range(len(labels)):
            child_node_id = node_id+'.'+str(i)
            child_node = self._build_tree(X_child[i], y_child[i],
                             depth=depth+1,
                             node_id=child_node_id,
                             parent_target_value = target_value)
            node.children[labels[i]] = child_node

        return node

    def _find_target_value(self, y):
        """Get the class label of highest frequency."""
        unique, counts = np.unique(y, return_counts=True)
        # Find the index of the maximum count
        max_index = np.argmax(counts)
        mode_value = unique[max_index]
        # mode_count = counts[max_index]
        return mode_value

    def get_best_splitter(self, X, y):
        max_gain = float("-inf")
        best_splitter = None
        for _ in range(self.n_splitters):
            splitter = self.get_candidate_splitter(X, y)
            labels = list(splitter[0].keys())
            measure = list(splitter[1].keys())[0]
            y_subs = [[] for k in range(len(labels))]
            for j in range(X.shape[0]):
                min_dist = float("inf")
                sub = None
                for k in range(len(labels)):
                    dist = distance(
                        X[j],
                        splitter[0][labels[k]],
                        metric=measure,
                        kwargs=splitter[1][measure],
                    )
                    if dist < min_dist:
                        min_dist = dist
                        sub = k
                y_subs[sub].append(y[j])
            y_subs = [np.array(ele) for ele in y_subs]
            gini_index = gini_gain(y, y_subs)
            if gini_index > max_gain:
                max_gain = gini_index
                best_splitter = splitter
        return best_splitter

    def _fit(self, X, y):
        # Set the unique class labels
        self.classes_ = list(np.unique(y))
        
        self.root = self._build_tree(
            X, y, depth=0, node_id="0", parent_target_value=None
        )
        self._is_fitted = True

    def _predict(self, X):
        if not self._is_fitted:
            raise NotFittedError(
                f"This instance of {self.__class__.__name__} has not "
                f"been fitted yet; please call `fit` first."
            )
        predictions = list()
        for i in range(len(X)):
            prediction = self.classify(self.root, X[i])
            predictions.append(prediction)
        predictions = np.array(predictions)
        return predictions

    def classify(self, treenode, x):
        # Classify one data point using the proximity tree
        if treenode._is_leaf:
            return treenode.label
        else:
            measure = list(treenode.splitter[1].keys())[0]
            branches = list(treenode.splitter[0].keys())
            min_dist = np.inf
            id = None
            for i in range(len(branches)):
                dist = distance(x, treenode.splitter[0][branches[i]], metric=measure, kwargs=treenode.splitter[1][measure])
                if dist < min_dist:
                    min_dist = dist
                    id = i
            return self.classify(treenode.children[branches[id]], x)


    def _predict_proba(self, X):
        if not self._is_fitted:
            raise NotFittedError(
                f"This instance of {self.__class__.__name__} has not "
                f"been fitted yet; please call `fit` first."
            )
        # Get the unique class labels
        classes = self.classes_
        class_count = len(classes)
        probas = []

        for i in range(len(X)):
            # Classify the data point and find the leaf node
            leaf_node = self._find_leaf(self.root, X[i])
        
            # Create probability distribution based on class counts in the leaf node
            proba = np.zeros(class_count)
            for class_label, class_proba in leaf_node.class_distribution.items():
                proba[classes.index(class_label)] = class_proba
            probas.append(proba)

        return np.array(probas)

    def _find_leaf(self, treenode, x):
        # Helper function to find the leaf node for a given data point
        if treenode._is_leaf:
            return treenode
        else:
            measure = list(treenode.splitter[1].keys())[0]
            branches = list(treenode.splitter[0].keys())
            min_dist = np.inf
            id = None
            for i in range(len(branches)):
                dist = distance(x, treenode.splitter[0][branches[i]], metric=measure, kwargs=treenode.splitter[1][measure])
                if dist < min_dist:
                    min_dist = dist
                    id = i
            return self._find_leaf(treenode.children[branches[id]], x)

    def _print(self):

        # Print the decision tree

        self._recursive_print(self.root)
    
    def _recursive_print(self, node):

        # Recursively print tree from node
        # node: TreeNode

        print(node)
        for child in node.children:
            self._recursive_print(node.children[child])

@njit(cache=True, fastmath=True)
def gini(y) -> float:
    """Get gini score at a specific node.

    Parameters
    ----------
    y : 1d numpy array
        array of class labels

    Returns
    -------
    score : float
        gini score for the set of class labels (i.e. how pure they are). A
        larger score means more impurity. Zero means
        pure.
    """
    # get number instances at node
    n_instances = y.shape[0]
    if n_instances > 0:
        # count each class
        unique_labels = list(np.unique(y))
        class_counts = []
        for i in range(len(unique_labels)):
            cnt = 0
            for j in range(len(y)):
                if y[j] == unique_labels[i]:
                    cnt += 1
            class_counts.append(cnt)
        class_counts = np.array(class_counts)
        # subtract class entropy from current score for each class
        class_counts = np.divide(class_counts, n_instances)
        class_counts = np.power(class_counts, 2)
        sum = np.sum(class_counts)
        return 1 - sum
    else:
        # y is empty, therefore considered pure
        raise ValueError("y empty")


#@njit(cache=True, fastmath=True)
def gini_gain(y, y_subs) -> float:
    """Get gini score of a split, i.e. the gain from parent to children.

    Parameters
    ----------
    y : 1d array
        array of class labels at parent
    y_subs : list of 1d array like
        list of array of class labels, one array per child

    Returns
    -------
    score : float
        gini score of the split from parent class labels to children. Note a
        higher score means better gain,
        i.e. a better split
    """
    # find number of instances overall
    parent_n_instances = y.shape[0]
    # if parent has no instances then is pure
    if parent_n_instances == 0:
        for child in y_subs:
            if len(child) > 0:
                raise ValueError("children populated but parent empty")
        return 0.5
    # find gini for parent node
    score = gini(y)
    # sum the children's gini scores
    for index in range(len(y_subs)):
        child_class_labels = y_subs[index]
        # ignore empty children
        if len(child_class_labels) > 0:
            # find gini score for this child
            child_score = gini(child_class_labels)
            # weight score by proportion of instances at child compared to
            # parent
            child_size = len(child_class_labels)
            child_score *= child_size / parent_n_instances
            # add to cumulative sum
            score -= child_score
    return score


In [3]:
clf = ProximityTree(n_splitters = 7, max_depth=6)

In [4]:
best_splitter = clf.get_best_splitter(X,y)

In [5]:
best_splitter

[{'1': array([-1.0736609 , -1.3838835 , -1.5971615 , -1.6165504 , -1.5971615 ,
         -1.4420502 , -0.99610534, -0.10421542,  0.72950777,  1.0978971 ,
          1.117286  ,  0.98156361,  0.76828559,  0.38050738,  0.39989629,
          0.55500757,  0.4192852 ,  0.67134104,  1.2142305 ,  1.0785082 ,
          0.61317431,  0.28356281,  0.01211805, -0.51138255]),
  '2': array([-0.73670703, -1.0028086 , -1.4167443 , -1.3576106 , -1.4463111 ,
         -1.4463111 , -1.6237121 , -0.73670703,  0.41639962,  1.1260037 ,
          1.2738379 ,  1.2442711 ,  1.1260037 ,  0.74163483,  0.80076851,
          1.0964369 ,  1.0373032 ,  0.65293432,  0.44596646,  0.32769911,
          0.03203074, -0.26363763,  0.0024639 , -0.29320447])},
 {'euclidean': {}}]

In [6]:
clf._fit(X,y)

In [7]:
root = clf._build_tree(X,y, depth=0, node_id='0')
print(root.children)

{'1': 0.0 Internal node splits using distance measure {'wdtw': {'g': 0.781}} and the branches are ['1', '2'], '2': 0.1 Internal node splits using distance measure {'msm': {'c': 10.0}} and the branches are ['1', '2']}


In [8]:
clf._print()

0 Internal node splits using distance measure {'ddtw': {'window': 0.053}} and the branches are ['1', '2']
0.0 Internal node splits using distance measure {'euclidean': {}} and the branches are ['1', '2']
0.0.0 Internal node splits using distance measure {'ddtw': {'window': 0.008}} and the branches are ['1', '2']
0.0.0.0 Internal node splits using distance measure {'wdtw': {'g': 0.217}} and the branches are ['1', '2']
0.0.0.0.0 Leaf node with label 1 and class distribution {'1': 1.0}
0.0.0.0.1 Internal node splits using distance measure {'dtw': {'window': 0.234}} and the branches are ['1', '2']
0.0.0.0.1.0 Leaf node with label 1 and class distribution {'1': 1.0}
0.0.0.0.1.1 Internal node splits using distance measure {'euclidean': {}} and the branches are ['1', '2']
0.0.0.0.1.1.0 Leaf node with label 1 and class distribution {'1': 1.0}
0.0.0.0.1.1.1 Leaf node with label 1 and class distribution {'1': 0.5, '2': 0.5}
0.0.0.1 Leaf node with label 1 and class distribution {'1': 0.5, '2': 0.

In [9]:
y_pred = clf._predict(X)
y_pred

array(['1', '1', '2', ..., '1', '2', '2'], dtype='<U1')

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y)

0.968065693430657

In [11]:
y_probs = clf._predict_proba(X)

In [12]:
y_probs

array([[1.        , 0.        ],
       [0.91489362, 0.08510638],
       [0.        , 1.        ],
       ...,
       [0.5       , 0.5       ],
       [0.        , 1.        ],
       [0.        , 1.        ]])

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, test_size=0.25)

In [50]:
clf = ProximityTree(n_splitters = 5, max_depth = 7)

In [14]:
clf._fit(X_train, y_train)

In [15]:
y_pred = clf._predict(X_test)
score = accuracy_score(y_test,y_pred)
score

0.9306569343065694